In [1]:
import numpy as np
from tqdm import *
from astropy.io import fits
from skimage import exposure

try:
    import cPickle as pickle
except ImportError:  # python 3.x
    import pickle


Define simualtion data names

In [2]:
download = False
if download:
    !wget -O dens.b1_p01_n1.fits https://faun.rc.fas.harvard.edu/bburkhart/b1p.01_512/dens_t500.fits.gz --no-check-certificate
    !wget -O dens.b1_p01_n2.fits https://faun.rc.fas.harvard.edu/bburkhart/b1p.01_512/dens_t550.fits.gz --no-check-certificate
    !wget -O dens.b1_p01_n3.fits https://faun.rc.fas.harvard.edu/bburkhart/b1p.01_512/dens_t600.fits.gz --no-check-certificate
    !wget -O dens.b1_p01_n4.fits https://faun.rc.fas.harvard.edu/bburkhart/b1p.01_512/dens_t650.fits.gz --no-check-certificate
    !wget -O dens.b1_p01_n5.fits https://faun.rc.fas.harvard.edu/bburkhart/b1p.01_512/dens_t700.fits.gz --no-check-certificate
    
    !wget -O dens.b01_p01_n1.fits https://faun.rc.fas.harvard.edu/bburkhart/b.1p.01_512/dens_t500.fits.gz --no-check-certificate
    !wget -O dens.b01_p01_n2.fits https://faun.rc.fas.harvard.edu/bburkhart/b.1p.01_512/dens_t550.fits.gz --no-check-certificate
    !wget -O dens.b01_p01_n3.fits https://faun.rc.fas.harvard.edu/bburkhart/b.1p.01_512/dens_t600.fits.gz --no-check-certificate
    !wget -O dens.b01_p01_n4.fits https://faun.rc.fas.harvard.edu/bburkhart/b.1p.01_512/dens_t650.fits.gz --no-check-certificate
    !wget -O dens.b01_p01_n5.fits https://faun.rc.fas.harvard.edu/bburkhart/b.1p.01_512/dens_t700.fits.gz --no-check-certificate


In [3]:
### note that we do not use n4, the 6.5 eddy turnover time snapshot

# super-alfvénic
sub_alfvens = ['dens.b1_p01_n1.fits', 'dens.b1_p01_n2.fits', 
               'dens.b1_p01_n3.fits', 'dens.b1_p01_n4.fits', 'dens.b1_p01_n5.fits']

# sub-alfvénic
sup_alfvens = ['dens.b01_p01_n1.fits', 'dens.b01_p01_n2.fits', 
               'dens.b01_p01_n3.fits', 'dens.b01_p01_n4.fits', 'dens.b01_p01_n5.fits']


In [4]:
FFP = True
ffpname = ''
if FFP:
    ffpname='.ffp'

In [5]:
# code to extract the images
def makeimages(data, imsize, nx, sh, nsh, idstem, filepath, saveimagesindividually=False, killpwr=False):
    imsize = int(imsize)
    dsh = data.shape
    newdata = data
    cube = np.zeros([imsize, imsize, int(dsh[2]*nsh*nsh*nx*nx)])
    iterct = 0
    IDs = []
    for s in np.arange(dsh[2]):
        if killpwr:
            img = data[:, :, s]
            fftsl= np.fft.fft2(img)
            normalized = fftsl/np.abs(fftsl)
            newimg = np.real(np.fft.ifft2(normalized))
            newdata[:, :, s] = newimg
    for s in tqdm(np.arange(dsh[2])):
        for r in np.arange(nsh):
            for q in np.arange(nsh):
                fullimg = np.roll(np.roll(newdata[:, :, s], int(r*sh+s), axis=0), int(q*sh+s), axis=1)
                for i in np.arange(nx):
                    for j in np.arange(nx):
                        toimg = fullimg[int(i*imsize):int((i+1)*imsize), int(j*imsize):int((j+1)*imsize)]
                        exeq = np.reshape(exposure.equalize_hist(toimg), (imsize, imsize, 1))
                        cube[:, :, iterct] = exeq[:, :, 0]
                        imgid = idstem+str(iterct)
                        IDs.append(imgid)
                        if saveimagesindividually:
                            np.save(filepath +'/' + imgid+'.npy', exeq)
                        iterct = iterct +1
                        #exeq = exeq.reshape([imsize, imsize, 1])*np.ones([imsize, imsize, 3])*255
                        #img = Image.fromarray(exeq.astype('int8'), mode='RGB')
    return cube, IDs

Some basic parameters for image extraction

In [6]:
# The size of the images to make
imsize = 128
# The "striding"
nsh = 2
# The stride size
sh = imsize/nsh
# The number of images across a cube
nx = 4

Extract the images for the sub-Alfvénic simulation

In [7]:
train_ids = []
test_ids = []
train_labels = []
test_labels = []

In [8]:
#Make the trainig data for sub-alfvenic case
for fn in sub_alfvens[0:3]:
    cube = fits.getdata(fn)
    data = np.log10(cube)
    data = (data- np.nanmin(data))/(np.nanmax(data)- np.nanmin(data))*255.0
    shcube = data.shape
    data[~np.isfinite(data)] = 0.

    train_x, ids= makeimages(data, int(imsize), nx, sh, nsh, 'subalf1_train'+ffpname, 'data', saveimagesindividually=True, killpwr=FFP)
    train_ids=train_ids+ids
    train_labels = train_labels+list(np.zeros(len(ids), dtype='int'))


100%|██████████| 512/512 [01:19<00:00,  6.73it/s]


Extract the images for the super-Alfvénic simulation

In [9]:
#Make the training data for super-alfvenic
for fn in sup_alfvens[0:3]:
    cube = fits.getdata(fn)
    data = np.log10(cube)
    data = (data- np.nanmin(data))/(np.nanmax(data)- np.nanmin(data))*255.0
    shcube = data.shape
    data[~np.isfinite(data)] = 0.

    train_x, ids= makeimages(data, int(imsize), nx, sh, nsh, 'supalf1_train'+ffpname, 'data', saveimagesindividually=True, killpwr=FFP)
    train_ids=train_ids+ids
    train_labels = train_labels+list(np.ones(len(ids), dtype='int'))



100%|██████████| 512/512 [01:19<00:00,  6.55it/s]


In [10]:
#Make the test data for sub-alfvenic case
for fn in sub_alfvens[4:]:
    cube = fits.getdata(fn)
    data = np.log10(cube)
    data = (data- np.nanmin(data))/(np.nanmax(data)- np.nanmin(data))*255.0
    shcube = data.shape
    data[~np.isfinite(data)] = 0.

    test_x, ids= makeimages(data, int(imsize), nx, sh, nsh, 'subalf1_test'+ffpname, 'data', saveimagesindividually=True, killpwr=FFP)
    test_ids=test_ids+ids
    test_labels = test_labels+list(np.zeros(len(ids), dtype='int'))


100%|██████████| 512/512 [01:21<00:00,  7.09it/s]


In [11]:
#Make the test data for super-alfvenic
for fn in sup_alfvens[4:]:
    cube = fits.getdata(fn)
    data = np.log10(cube)
    data = (data- np.nanmin(data))/(np.nanmax(data)- np.nanmin(data))*255.0
    shcube = data.shape
    data[~np.isfinite(data)] = 0.

    test_x, ids= makeimages(data, int(imsize), nx, sh, nsh, 'supalf1_test'+ffpname, 'data', saveimagesindividually=True, killpwr=FFP)
    test_ids=test_ids+ids
    test_labels = test_labels+list(np.ones(len(ids), dtype='int'))



100%|██████████| 512/512 [01:18<00:00,  6.69it/s]


In [12]:
all_labels = train_labels+test_labels
all_ids = train_ids + test_ids
zip_label_ids = zip(all_ids, all_labels)
labels = dict(zip_label_ids)
partition = {'train': train_ids, 'validation': test_ids}

In [13]:
with open('labels5_5.p', 'wb') as fp:
    pickle.dump(labels, fp, protocol=pickle.HIGHEST_PROTOCOL)

with open('partition5_5.p', 'wb') as fp:
    pickle.dump(partition, fp, protocol=pickle.HIGHEST_PROTOCOL)